In [1]:
%cd ~/Dev/video-membership

/Users/cfe/Dev/video-membership


In [2]:
from app import db
from app.playlists.models import Playlist
from app.users.models import User
from app.videos.models import Video

from cassandra.cqlengine.management import sync_table

In [3]:
from app.indexing.schemas import *

In [4]:
db.get_session()
sync_table(User)
sync_table(Video)
sync_table(Playlist)

In [5]:
import uuid
import json
from pydantic import BaseModel, Field, validator, root_validator
from typing import Optional

In [6]:
video_obj = dict(Video.objects.first())
video_obj

{'host_id': 'PmZ29Vta7Vc',
 'db_id': UUID('fa0cac06-6034-11ec-9f47-acde48001122'),
 'host_service': 'youtube',
 'title': 'OpenCV Is Another great Package',
 'url': 'https://www.youtube.com/watch?v=PmZ29Vta7Vc&t=3250s',
 'user_id': UUID('9301d5ea-5544-11ec-b5af-acde48001122')}

In [7]:
class VideoIndexSchema2(BaseModel):
    objectID: str = Field(alias='host_id')
    objectType: str = "Video"
    title: Optional[str]
    path: str = Field(alias='host_id')
        
    @validator("path")
    def set_path(cls, v, values, **kwargs):
        host_id = v
        return f"/videos/{host_id}"

In [8]:
VideoIndexSchema(**video_obj).dict()

{'objectID': 'PmZ29Vta7Vc',
 'objectType': 'Video',
 'title': 'OpenCV Is Another great Package',
 'path': '/videos/PmZ29Vta7Vc'}

In [9]:
playlist_obj = dict(Playlist.objects.first())
playlist_obj

{'db_id': UUID('c52b68ca-6024-11ec-8b94-acde48001122'),
 'user_id': None,
 'updated': datetime.datetime(2021, 12, 20, 17, 35, 31, 236000),
 'host_ids': [],
 'title': 'hello world'}

In [10]:
class PlaylistIndexSchema2(BaseModel):
    objectID: uuid.UUID = Field(alias='db_id')
    objectType: str = "Playlist"
    title: Optional[str]
    path: str = Field(default='/')
    
    @root_validator
    def set_defaults(cls, values):
        objectID = values.get('objectID')
        values['objectID'] = str(objectID)
        values['path'] = f"/playlists/{objectID}"
        return values

In [11]:
# json.loads(PlaylistIndexSchema(**playlist_obj).json())

In [12]:
PlaylistIndexSchema(**playlist_obj).dict()

{'objectID': 'c52b68ca-6024-11ec-8b94-acde48001122',
 'objectType': 'Playlist',
 'title': 'hello world',
 'path': '/playlists/c52b68ca-6024-11ec-8b94-acde48001122'}

In [13]:
video_q = [dict(x) for x in Video.objects.all()]
videos_dataset = [VideoIndexSchema(**x).dict() for x in video_q]
videos_dataset

[{'objectID': 'PmZ29Vta7Vc',
  'objectType': 'Video',
  'title': 'OpenCV Is Another great Package',
  'path': '/videos/PmZ29Vta7Vc'},
 {'objectID': 'zgQ4WFGMgRM',
  'objectType': 'Video',
  'title': 'Yet another one!',
  'path': '/videos/zgQ4WFGMgRM'},
 {'objectID': '1eHQIu4r0Bc',
  'objectType': 'Video',
  'title': 'OpenCV Is Awesome!',
  'path': '/videos/1eHQIu4r0Bc'},
 {'objectID': '5agh7ucYKB4',
  'objectType': 'Video',
  'title': 'Try Django again',
  'path': '/videos/5agh7ucYKB4'},
 {'objectID': 'zf1sDqym8ik',
  'objectType': 'Video',
  'title': 'Try Django 2',
  'path': '/videos/zf1sDqym8ik'},
 {'objectID': '9ix7TUGVYIo',
  'objectType': 'Video',
  'title': None,
  'path': '/videos/9ix7TUGVYIo'},
 {'objectID': 'kd_c99cU0SM',
  'objectType': 'Video',
  'title': 'My title',
  'path': '/videos/kd_c99cU0SM'},
 {'objectID': '4cJpiOPKH14',
  'objectType': 'Video',
  'title': 'Star Wars Eclipse – Official Cinematic Reveal Trailer',
  'path': '/videos/4cJpiOPKH14'},
 {'objectID': 'PF3iy

In [14]:
playlist_q = [dict(x) for x in Playlist.objects.all()]
playlists_dataset = [PlaylistIndexSchema(**x).dict() for x in playlist_q]
playlists_dataset

[{'objectID': 'c52b68ca-6024-11ec-8b94-acde48001122',
  'objectType': 'Playlist',
  'title': 'hello world',
  'path': '/playlists/c52b68ca-6024-11ec-8b94-acde48001122'},
 {'objectID': 'e3a72492-5ea3-11ec-bfdb-acde48001122',
  'objectType': 'Playlist',
  'title': 'hello world',
  'path': '/playlists/e3a72492-5ea3-11ec-bfdb-acde48001122'},
 {'objectID': '312794ce-5f5b-11ec-8696-acde48001122',
  'objectType': 'Playlist',
  'title': 'Creator Videos',
  'path': '/playlists/312794ce-5f5b-11ec-8696-acde48001122'},
 {'objectID': '1b56e168-5f5b-11ec-a4c2-acde48001122',
  'objectType': 'Playlist',
  'title': 'Video Trailers',
  'path': '/playlists/1b56e168-5f5b-11ec-a4c2-acde48001122'}]

In [15]:
dataset = playlists_dataset + videos_dataset

In [16]:
dataset

[{'objectID': 'c52b68ca-6024-11ec-8b94-acde48001122',
  'objectType': 'Playlist',
  'title': 'hello world',
  'path': '/playlists/c52b68ca-6024-11ec-8b94-acde48001122'},
 {'objectID': 'e3a72492-5ea3-11ec-bfdb-acde48001122',
  'objectType': 'Playlist',
  'title': 'hello world',
  'path': '/playlists/e3a72492-5ea3-11ec-bfdb-acde48001122'},
 {'objectID': '312794ce-5f5b-11ec-8696-acde48001122',
  'objectType': 'Playlist',
  'title': 'Creator Videos',
  'path': '/playlists/312794ce-5f5b-11ec-8696-acde48001122'},
 {'objectID': '1b56e168-5f5b-11ec-a4c2-acde48001122',
  'objectType': 'Playlist',
  'title': 'Video Trailers',
  'path': '/playlists/1b56e168-5f5b-11ec-a4c2-acde48001122'},
 {'objectID': 'PmZ29Vta7Vc',
  'objectType': 'Video',
  'title': 'OpenCV Is Another great Package',
  'path': '/videos/PmZ29Vta7Vc'},
 {'objectID': 'zgQ4WFGMgRM',
  'objectType': 'Video',
  'title': 'Yet another one!',
  'path': '/videos/zgQ4WFGMgRM'},
 {'objectID': '1eHQIu4r0Bc',
  'objectType': 'Video',
  'titl